## Pulling tweet data for twitter user analysis

I want to see how different peoples' follows behave differently and if it correlates with their experience 



In [ ]:
import pandas as pd
from twarc import Twarc2, expansions
from twarc_csv import DataFrameConverter, CSVConverter


In [ ]:
## Get dates for a 24 hour window to pass to twarc2 timeline command
from datetime import datetime, timezone, timedelta

def get_time_interval(hours=24):
    """
    Return EndTime, StartTime, from current time
    """
    now = datetime.now(timezone.utc)
    (now - timedelta(hours=24)).isoformat("T")[:-3] + "Z"
    return now.isoformat("T")[:-13]+"Z",  (now - timedelta(hours=24)).isoformat("T")[:-13] + "Z"


#YYYY-MM-DDTHH:mm:ssZ

In [ ]:
end_time, start_time = get_time_interval()
end_time

In [ ]:
## Get a users following users

USER = "nicktorba" 

current_time, _ = get_time_interval()

FOLLOWING_DIR = f"{USER}/following"

client = Twarc2(
    consumer_key="Zz4fBlLrWc8VWAqpN3BNUnjHx", 
    consumer_secret="A60T0PHokfGVO2yWyMjEm0uy4nl0zX5l3RLwOa8sN703Ir7UK1",
    access_token="1381389740616908801-R2eZlK4Q8LoVOCz9XcfLhOf24gf1hp", 
    access_token_secret="kVYukPdOOfDyb85O3dO3feyivWlRmYZkY93EaErV2OIPS"
)

res = client.following("nicktorba")
res = next(res)
df_following = DataFrameConverter("users").process(res["data"])
print(df_following.shape)
df_following.to_csv(f"{FOLLOWING_DIR}/following-{end_time}.csv")

In [ ]:
df = None
converter = DataFrameConverter("tweets", inline_referenced_tweets=True) #includes "included" tweets, aka referenced tweets
for id_ in df_following["id"].tolist():
    timeline_gen = client.timeline(id_, start_time=start_time, end_time=end_time)
    for res in timeline_gen:
        df_next = converter.process([res])
        if df is None:
            df = df_next
        else: 
            df = pd.concat([df, df_next])

data_fpath = f"nicktorba/tweets/following-tweets-{start_time}--{end_time}.csv"
print(df.shape)
df.to_csv(data_fpath)

In [ ]:
# !twarc2 timelines nicktorba/following-ids.txt nicktorba/tweets/following-tweets-2022-06-06015:22:33.932965+00:00--2022-06-07-15:22:33.932965+00:00V222.json --start-time 2022-06-06T15:22:33


In [ ]:
import json 
with open("nicktorba/tweets/following-tweets-2022-06-06015:22:33.932965+00:00--2022-06-07-15:22:33.932965+00:00V222.json", "r") as f:
    d = f.readlines()

d = json.loads(d[0])
d["data"][0]

In [ ]:
d["includes"]["users"]

In [ ]:
## Convert tweet data to csv with twarc2 csv plugin
# !twarc2 csv nicktorba/tweets/following-tweets-2022-06-06015:22:33.932965+00:00--2022-06-07-15:22:33.932965+00:00V222.json nicktorba/tweets/following-tweets-2022-06-06015:22:33.932965+00:00--2022-06-07-15:22:33.932965+00:00V222.csv

In [ ]:
### Load Data 
# df = pd.read_csv("nicktorba/tweets/following-tweets-2022-06-06T07:49:51-2022-06-07T07:49:51.csv")
df = pd.read_csv(data_fpath)
df["tweet_link"] = df.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)
df.loc[:, "created_at"] = pd.to_datetime(df.loc[:, "created_at"], utc=True)
df["created_at.hour"] = df["created_at"].dt.floor('h')


In [ ]:
min_time = Timestamp('2022-06-13 17:37:03+0000', tz='UTC')


In [ ]:
from twarc import Twarc2, expansions
import json

# This is where you initialize the client with your own bearer token (replace the XXXXX with your own bearer token)
bearer_token = "AAAAAAAAAAAAAAAAAAAAAOIvZwEAAAAAAvVNdH%2FfRszdvwhjzFHRXco6Exc%3DoMJQzs2cOS0GNZUJZVAWOCONnYgR6ENCl41qb2pRI00DvMLzyo"
client = Twarc2(consumer_key="Zz4fBlLrWc8VWAqpN3BNUnjHx", 
                consumer_secret="A60T0PHokfGVO2yWyMjEm0uy4nl0zX5l3RLwOa8sN703Ir7UK1",
                access_token="1381389740616908801-R2eZlK4Q8LoVOCz9XcfLhOf24gf1hp", 
                access_token_secret="kVYukPdOOfDyb85O3dO3feyivWlRmYZkY93EaErV2OIPS"
)

query = "to:deepfates"

r = client.search_recent(query=query, start_time="2022-06-13T17:37:03Z") #YYYY-MM-DDTHH:mm:ssZ
                                                    

In [ ]:
import pandas as pd 

results = []

res = next(r)

# for i in r:
#     results.append(i)
    

In [ ]:
res["data"]

In [ ]:
len(res["data"])

In [ ]:
res.keys()

In [ ]:
len(res["includes"]["tweets"])

In [ ]:
from twarc_csv import DataFrameConverter

df = DataFrameConverter("tweets").process(res["data"])
df_i = DataFrameConverter("tweets").process(res["includes"]["tweets"])
df_users = DataFrameConverter("users").process(res["includes"]["users"])

df_users.head()

In [ ]:
res["includes"]["users"]

In [ ]:
import twarc_csv

dir(twarc_csv)